# Intel & MobileODT Cervical Cancer Screening
* [Intel & MobileODT Cervical Cancer Screening | Kaggle](https://www.kaggle.com/c/intel-mobileodt-cervical-cancer-screening/data)

In [1]:
import numpy as np
import pandas as pd

## Data Resized

In [2]:
#path = "/home/tsu-nera/repo/kaggle/input/intel-mobileodt-cervical-cancer-screening/"
path="/input/"
width = 100
height = 100

In [ ]:
import os
os.makedirs(path+"resized", exist_ok=True)

for filetype in ["train/", "valid/"]:
    for category in ["Type_1", "Type_2", "Type_3"]:
        os.makedirs(path+"resized/"+filetype+category,  exist_ok=True)
os.makedirs(path+"resized/test/unknown",  exist_ok=True)

In [ ]:
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
def resize_img(src, dst, width, height):
    img = Image.open(src, 'r')
    resized = img.resize((width, height))
    resized.save(dst, 'JPEG', optimize=True)

In [ ]:
%cd $path

In [ ]:
import glob
categories = ['Type_1/', 'Type_2/', 'Type_3/']
filetypes = ["train/"]

for filetype in filetypes:
    for category in categories:
            globpath = filetype+category+"*.jpg"
            files = glob.glob(globpath)
            for file in files:
                src = path+file
                dst = path+"resized/"+file
                resize_img(src, dst, width, height)

In [ ]:
%cd $path

In [ ]:
globpath = "test/unknown/*.jpg"
files = glob.glob(globpath)
for file in files:
    src = path+file
    dst = path+"resized/"+file
    resize_img(src, dst, width, height)

##  Create train/valid data

In [ ]:
path = "/home/tsu-nera/repo/kaggle/input/intel-mobileodt-cervical-cancer-screening/"

In [ ]:
%cd $path

In [ ]:
%mkdir -p valid/Type_1
%mkdir -p valid/Type_2
%mkdir -p valid/Type_3

In [ ]:
%cd $path
%cd resized/train

In [ ]:
import glob
import shutil
g = glob.glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(81): 
    shutil.move(shuf[i], '../valid/' + shuf[i])

## Load Data

In [3]:
import keras
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [4]:
batch_size = 64

In [5]:
gen_t = image.ImageDataGenerator(rescale=1. / 255,
                 rotation_range=15, height_shift_range=0.05, 
                 shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
gen = image.ImageDataGenerator(rescale=1. / 255)

In [6]:
batches = gen_t.flow_from_directory(path+'train', batch_size=batch_size, target_size=(width,height))
val_batches = gen.flow_from_directory(path+'valid', batch_size=batch_size, target_size=(width, height))
test_batches = gen.flow_from_directory(path+'test', batch_size=batch_size, target_size=(width, height))

Found 500 images belonging to 3 classes.
Found 81 images belonging to 3 classes.
Found 512 images belonging to 1 classes.


## Use Vgg16

In [7]:
import keras
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam, Nadam
from keras.applications.vgg16 import VGG16
from keras.layers.advanced_activations import PReLU

In [8]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(width,height,3))

###  finetune
* [Fine-tuning pre-trained VGG16 not possible since `add` method is not defined for `Model` class? · Issue #4040 · fchollet/keras](https://github.com/fchollet/keras/issues/4040)

In [9]:
nf = 128
p = 0.5

In [10]:
base_model.output_shape
last = base_model.output

In [11]:
from keras import backend as K
K.set_image_dim_ordering('th')

x = BatchNormalization(axis=1)(last)
x = Conv2D(nf,3,3, border_mode='same')(x)
x = PReLU()(x)
x = BatchNormalization(axis=1)(x)
#x = MaxPooling2D(dim_ordering="th")(x)
x = Conv2D(nf,3,3, border_mode='same')(x)
x = PReLU()(x)
x = BatchNormalization(axis=1)(x)
#x = MaxPooling2D(dim_ordering="th")(x)
x = Conv2D(nf,3,3, border_mode='same')(x)
x = PReLU()(x)
x = BatchNormalization(axis=1)(x)
x = MaxPooling2D((1,2))(x)
x = Conv2D(3,3,3, border_mode='same')(x)
x = Dropout(p)(x)
x = GlobalAveragePooling2D()(x)
preds = Activation('softmax')(x)

model = Model(base_model.input, preds)

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same")`
  """
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same")`
  if __name__ == '__main__':
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same")`
  del sys.path[0]
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (3, 3), padding="same")`


In [12]:
for layer in base_model.layers:
    layer.trainable = False

In [13]:
model.compile(Nadam(), loss='categorical_crossentropy', metrics=['accuracy'])

大量のWarningがでてkernel が死ぬときの解決法
* [Intel & MobileODT Cervical Cancer Screening | Kaggle](https://www.kaggle.com/c/intel-mobileodt-cervical-cancer-screening/discussion/31558)

これでも死んだ。画像サイズがおおきいのかな？？ リサイズしてみる。

In [14]:
from keras.callbacks import EarlyStopping
escb = EarlyStopping(monitor="loss", patience=30)

In [15]:
model.fit_generator(batches, batches.n//batch_size, nb_epoch=100,
                    validation_data=val_batches, validation_steps=val_batches.n//batch_size, callbacks=[escb])

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., 7, epochs=100, validation_data=<keras.pre..., validation_steps=1, callbacks=[<keras.ca...)`
  


Epoch 1/100
7/7 [==============================] - 12s - loss: 1.2219 - acc: 0.4978 - val_loss: 1.1085 - val_acc: 0.1250
Epoch 2/100
7/7 [==============================] - 9s - loss: 1.0213 - acc: 0.5376 - val_loss: 1.0727 - val_acc: 0.5882
Epoch 3/100
7/7 [==============================] - 7s - loss: 0.9593 - acc: 0.5753 - val_loss: 0.9894 - val_acc: 0.7059
Epoch 4/100
7/7 [==============================] - 7s - loss: 0.9761 - acc: 0.5757 - val_loss: 1.0244 - val_acc: 0.5294
Epoch 5/100
7/7 [==============================] - 7s - loss: 0.9677 - acc: 0.5694 - val_loss: 0.9629 - val_acc: 0.5882
Epoch 6/100
7/7 [==============================] - 7s - loss: 0.9659 - acc: 0.5560 - val_loss: 1.0503 - val_acc: 0.5294
Epoch 7/100
7/7 [==============================] - 7s - loss: 0.9470 - acc: 0.5914 - val_loss: 0.9132 - val_acc: 0.5882
Epoch 8/100
7/7 [==============================] - 7s - loss: 0.9620 - acc: 0.5690 - val_loss: 0.9277 - val_acc: 0.7059
Epoch 9/100
7/7 [======================

7/7 [==============================] - 8s - loss: 0.0912 - acc: 0.9727 - val_loss: 7.5850 - val_acc: 0.5294
Epoch 69/100
7/7 [==============================] - 7s - loss: 0.1096 - acc: 0.9682 - val_loss: 7.5850 - val_acc: 0.5294
Epoch 70/100
7/7 [==============================] - 7s - loss: 0.0886 - acc: 0.9722 - val_loss: 6.6369 - val_acc: 0.5882
Epoch 71/100
7/7 [==============================] - 8s - loss: 0.1498 - acc: 0.9503 - val_loss: 11.3775 - val_acc: 0.2941
Epoch 72/100
7/7 [==============================] - 7s - loss: 0.1035 - acc: 0.9583 - val_loss: 8.5331 - val_acc: 0.4706
Epoch 73/100
7/7 [==============================] - 8s - loss: 0.0807 - acc: 0.9665 - val_loss: 7.5850 - val_acc: 0.5294
Epoch 74/100
7/7 [==============================] - 7s - loss: 0.1180 - acc: 0.9633 - val_loss: 7.5850 - val_acc: 0.5294
Epoch 75/100
7/7 [==============================] - 8s - loss: 0.0689 - acc: 0.9749 - val_loss: 5.6887 - val_acc: 0.6471
Epoch 76/100
7/7 [==========================

## Predict

In [17]:
preds = model.predict_generator(test_batches, test_batches.n//batch_size)

## Submit

In [18]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/2, mx)

In [19]:
subm = do_clip(preds,0.93)

In [20]:
test_filenames = [a[8:] for a in test_batches.filenames]

In [21]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image_name', test_filenames)
submission.head()

,image_name,Type_1,Type_2,Type_3
0,10.jpg,0.035,0.93,0.035
1,377.jpg,0.035,0.93,0.035
2,296.jpg,0.035,0.93,0.035
3,458.jpg,0.035,0.93,0.035
4,89.jpg,0.035,0.93,0.035


In [22]:
%cd '/home/tsu-nera/repo/kaggle/intel-mobileodt-cervical-cancer-screening'

[Errno 2] No such file or directory: '/home/tsu-nera/repo/kaggle/intel-mobileodt-cervical-cancer-screening'
/output


In [23]:
from IPython.display import FileLink
submission.to_csv('submission.csv', index=False)
FileLink('submission.csv')

/output/submission.csv